In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import math

In [2]:
review_distribution = pd.read_json('../full_dataset/visit_distribution_authorityid.json')
review_distribution['merge_token'] = review_distribution.apply(lambda x: x['authority_id'] + ',' + str(x['authority_stars']) + ',' + str(x['distance']), axis = 1)
review_distribution.head()

,0,authority_id,authority_stars,distance,time_distribution,merge_token
0,3,-_1ctLaz3jhPYc12hKXsEQ,1,1,0,"-_1ctLaz3jhPYc12hKXsEQ,1,1"
1,10,-_1ctLaz3jhPYc12hKXsEQ,2,1,0,"-_1ctLaz3jhPYc12hKXsEQ,2,1"
10,4,-_1ctLaz3jhPYc12hKXsEQ,3,3,0,"-_1ctLaz3jhPYc12hKXsEQ,3,3"
100,9,B5WkNWDxZ-baWoQc6DBNHA,1,2,0,"B5WkNWDxZ-baWoQc6DBNHA,1,2"
101,2,B5WkNWDxZ-baWoQc6DBNHA,2,2,0,"B5WkNWDxZ-baWoQc6DBNHA,2,2"


In [3]:
d_before = DataFrame(review_distribution.loc[review_distribution['time_distribution'] == 0])
d_before.columns=['count_before', 'authority_id', 'authority_stars', 'distance', 'time_distribution', 'merge_token']
d_after = DataFrame(review_distribution.loc[review_distribution['time_distribution'] == 1][['merge_token', '0']])
d_after.columns = ['merge_token', 'count_after']

In [4]:
factor = pd.merge(d_before, d_after, on = 'merge_token')[['authority_id', 'authority_stars', 'distance', 'count_before', 'count_after']]
factor['factor'] = factor.apply(lambda x: x['count_after'] / x['count_before'], axis = 1)
factor.head()

,authority_id,authority_stars,distance,count_before,count_after,factor
0,-_1ctLaz3jhPYc12hKXsEQ,2,1,10,5,0.500000
1,-_1ctLaz3jhPYc12hKXsEQ,3,3,4,34,8.500000
2,B5WkNWDxZ-baWoQc6DBNHA,1,2,9,3,0.333333
3,B5WkNWDxZ-baWoQc6DBNHA,2,2,2,20,10.000000
4,B5WkNWDxZ-baWoQc6DBNHA,4,2,21,67,3.190476


In [5]:
aggregate_result = factor[['authority_stars', 'distance', 'factor']].groupby(['authority_stars', 'distance'], as_index = False)['factor'].agg({'size': np.size, 'factor': np.mean, 'sigma': np.std })
aggregate_result.loc[aggregate_result['size'] > 3]

,authority_stars,distance,size,factor,sigma
1,1,2,8,2.214453,1.651443
2,1,3,9,3.787000,5.008333
3,1,4,5,1.311111,0.971771
4,1,5,4,1.437500,0.657489
6,2,2,9,2.351739,3.138038
7,2,3,5,2.385670,2.163669
11,3,2,15,2.689174,3.675056
12,3,3,13,8.037302,16.043247
13,3,4,10,9.617063,13.501548
15,4,1,9,1.923611,2.016964


In [6]:
factor['weight'] = factor.apply(lambda x: x['count_before'] + x['count_after'], axis = 1)
def weighted_avg_and_std(values):
    """
    Return the weighted average and biased weighted standard deviation.

    Adapted from: http://stackoverflow.com/questions/2413522/weighted-standard-deviation-in-numpy
    """
    
    average = np.average(values, weights=factor.loc[values.index, 'weight'])
    variance = np.average((values-average)**2, weights=factor.loc[values.index, 'weight'])  # Fast and numerically precise
    return (average, math.sqrt(variance))

In [7]:
agg_weighted = factor[['authority_stars', 'distance', 'factor', 'weight']].groupby(['authority_stars', 'distance'], as_index = False).agg({ 'factor': weighted_avg_and_std, 'weight': np.sum })
agg_weighted.loc[aggregate_result['size'] > 3]

,authority_stars,distance,weight,factor
1,1,2,375,"(2.02509186119, 1.3558989145456517)"
2,1,3,529,"(2.10996575713, 2.886231482255738)"
3,1,4,147,"(1.55922686535, 0.6664240037564244)"
4,1,5,15,"(1.28333333333, 0.590668171555645)"
6,2,2,287,"(1.61071103295, 2.547865232421651)"
7,2,3,308,"(1.41816158919, 0.8053650232386216)"
11,3,2,684,"(3.86021873712, 3.7877268675491775)"
12,3,3,907,"(9.05045763987, 14.25722737708027)"
13,3,4,297,"(17.6797872909, 15.389291169789296)"
15,4,1,93,"(2.3125, 1.5737898576776483)"
